References
- https://www.kaggle.com/code/huikang/arc-agi-2-code-approach
- https://www.kaggle.com/code/huikang/r1-distill-qwen-tir

```
uv run python3 kaggle.py
```

# Configuration

In [ ]:
serve_vllm_on_kaggle = True
run_all_questions = False   # ignored for submissions

In [ ]:
import os
import time
from kaggle_secrets import UserSecretsClient

secrets = UserSecretsClient()
REMOTE_VLLM_URL = "NOT_AVAILABLE"
if not serve_vllm_on_kaggle:
    REMOTE_VLLM_URL = secrets.get_secret("REMOTE_VLLM_URL")


start_time = time.time()
final_cutoff_time = start_time + (4 * 60 + 50) * 60  # 5 hours from start time


def is_on_kaggle_commit() -> bool:
    return os.getenv("KAGGLE_KERNEL_RUN_TYPE") == "Batch" and not bool(
        os.getenv("KAGGLE_IS_COMPETITION_RERUN")
    )


def is_on_kaggle_interactive() -> bool:
    return os.getenv("KAGGLE_KERNEL_RUN_TYPE") == "Interactive" and not bool(
        os.getenv("KAGGLE_IS_COMPETITION_RERUN")
    )


def is_on_kaggle() -> bool:
    return bool(os.getenv("KAGGLE_KERNEL_RUN_TYPE"))


# Some debugger warning on Kaggle
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"

In [ ]:
# print settings
print(f"{is_on_kaggle()=}")
print(f"{is_on_kaggle_interactive()=}")
print(f"{is_on_kaggle_commit()=}")
print(f"{serve_vllm_on_kaggle=}")
print(f"{run_all_questions=}")
print(f"{REMOTE_VLLM_URL[::-1][:13][::-1]=}")

# Setup

In [ ]:
import subprocess

if is_on_kaggle():
    subprocess.run(
        [
            "pip",
            "uninstall",
            "--yes",
            "tensorflow",
            "matplotlib",
            "keras",
            "scikit-learn",
        ]
    )

In [ ]:
import torch
import numpy as np


cutoff_times = [
    int(x) for x in np.linspace(final_cutoff_time, start_time + 15 * 60, 50 + 1)
]  # 5 minutes loading time at the start
cutoff_times.pop()

import shutil

if __name__ == "__main__" and os.path.exists("solutions"):
    shutil.rmtree("solutions")
os.makedirs("solutions", exist_ok=True)

if is_on_kaggle():
    if serve_vllm_on_kaggle:
        assert torch.cuda.is_available()
        assert torch.cuda.device_count() == 1
    else:
        # Check internet access is available when using remote inference
        import urllib.request
        from urllib.error import URLError

        try:
            urllib.request.urlopen("https://modal.com", timeout=5)
            print("Internet access confirmed")
        except (URLError, TimeoutError) as e:
            raise RuntimeError(
                "Internet access required when serve_vllm_on_kaggle=False"
            ) from e

        # Check that you are not wasting Kaggle GPUs
        assert not torch.cuda.is_available()
        assert torch.cuda.device_count() == 0

# Serve vLLM

In [ ]:
if is_on_kaggle():
    subprocess.run(["ls", "/kaggle/usr/lib/pip_install_aimo3_1/tiktoken_encodings"])

In [ ]:
with open("a-vllm.log", "w") as f:
    f.write("")

In [ ]:
import subprocess

num_generations = 6
max_model_len = 131072


def start_vllm_server() -> subprocess.Popen[bytes]:
    """Start vLLM server in the background"""
    os.environ["TRANSFORMERS_NO_TF"] = "1"
    os.environ["TRANSFORMERS_NO_FLAX"] = "1"
    os.environ["VLLM_ATTENTION_BACKEND"] = "TRITON_ATTN"
    os.environ["TRITON_PTXAS_PATH"] = "/usr/local/cuda/bin/ptxas"
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    # https://docs.vllm.ai/projects/recipes/en/latest/OpenAI/GPT-OSS.html#troubleshooting
    os.environ["TIKTOKEN_ENCODINGS_BASE"] = (
        "/kaggle/usr/lib/pip_install_aimo3_1/tiktoken_encodings"
    )

    command: list[str] = [
        "python",
        "-m",
        "vllm.entrypoints.openai.api_server",
        "--model",
        "/kaggle/input/gpt-oss-120b/transformers/default/1",
        "--served-model-name",
        "vllm-model",
        "--tensor-parallel-size",
        "1",
        "--max-num-seqs",
        f"{num_generations}",
        "--gpu-memory-utilization",
        "0.96",  # any higher may not have enough for graph capture
        "--host",
        "0.0.0.0",
        "--port",
        "8000",
        "--dtype",
        "auto",
        "--max-model-len",
        f"{max_model_len}",
    ]

    # Start the process in the background
    with open("/kaggle/working/a-vllm.log", "w") as logfile:
        process: subprocess.Popen[bytes] = subprocess.Popen(
            command, stdout=logfile, stderr=subprocess.STDOUT, start_new_session=True
        )

    print("Logs: /kaggle/working/a-vllm.log")
    return process


# Start the server
if is_on_kaggle() and serve_vllm_on_kaggle:
    vllm_process: subprocess.Popen[bytes] = start_vllm_server()

In [ ]:
import time


def await_client(printing: bool = False):
    for _ in range(15 * 60):
        time.sleep(1)
        try:
            model_list = client.models.list()
            if printing:
                print(model_list)
        except NameError:
            raise  # maybe you did not run the cell initializing client
        except Exception:
            continue
        break
    else:
        raise


if is_on_kaggle_interactive():
    # cannot await client on submission
    # because inference server needs to start within 15 minutes
    await_client()

# Token processing

In [ ]:
import os
from openai import OpenAI, Stream
from openai.types import Completion

# Point the client to vLLM server (local on Kaggle, Modal otherwise)
if is_on_kaggle() and serve_vllm_on_kaggle:
    os.environ["OPENAI_API_BASE"] = "http://127.0.0.1:8000/v1"
else:
    os.environ["OPENAI_API_BASE"] = REMOTE_VLLM_URL
    if is_on_kaggle():
        # openai_harmony uses TIKTOKEN_ENCODINGS_BASE to read pre-downloaded files
        os.environ["TIKTOKEN_ENCODINGS_BASE"] = (
            "/kaggle/usr/lib/pip_install_aimo3_1/tiktoken_encodings"
        )
os.environ["OPENAI_API_KEY"] = "sk-local"  # any non-empty string

client: OpenAI = OpenAI(
    base_url=os.environ["OPENAI_API_BASE"],
    api_key=os.environ["OPENAI_API_KEY"],
)

# Initialize openai-harmony encoding for GPT-OSS models
from openai_harmony import (
    Conversation,
    DeveloperContent,
    HarmonyEncodingName,
    Message,
    ReasoningEffort,
    Role,
    StreamableParser,
    SystemContent,
    load_harmony_encoding,
)

harmony_encoding = load_harmony_encoding(HarmonyEncodingName.HARMONY_GPT_OSS)
stop_token_ids: list[int] = list(harmony_encoding.stop_tokens_for_assistant_actions())

def build_prompt_token_ids(
    system_content: str,
    user_content: str,
    reasoning_effort: ReasoningEffort,
) -> list[int]:
    """Convert system and user content to token IDs using harmony format."""
    system_content_obj = SystemContent.new().with_reasoning_effort(reasoning_effort)
    system_message = Message.from_role_and_content(
        Role.SYSTEM,
        system_content_obj,
    )
    developer_message = Message.from_role_and_content(
        Role.DEVELOPER,
        DeveloperContent.new().with_instructions(system_content),
    )
    user_message = Message.from_role_and_content(
        Role.USER,
        user_content,
    )
    convo = Conversation.from_messages(
        [system_message, developer_message, user_message]
    )
    return list(
        harmony_encoding.render_conversation_for_completion(convo, Role.ASSISTANT)
    )


def append_user_turn_token_ids(
    prompt_ids: list[int], response_ids: list[int], user_content: str
) -> list[int]:
    """Append response token IDs and a new user turn to the prompt."""
    all_tokens = prompt_ids + response_ids
    # Build new user message and render to tokens
    new_user_message = Message.from_role_and_content(Role.USER, user_content)
    user_tokens = list(
        harmony_encoding.render_conversation_for_completion(
            Conversation.from_messages([new_user_message]), Role.ASSISTANT
        )
    )
    # Combine: previous prompt + response + user turn tokens
    return all_tokens + user_tokens

In [ ]:
from cachetools import cached, TTLCache
import os
import time
import requests


@cached(cache=TTLCache(maxsize=50, ttl=20))
def get_gpu_kv_cache_usage(question_id: str | None = None) -> float:
    # Parse vLLM /metrics endpoint using configured base URL
    try:
        base_url = os.environ["OPENAI_API_BASE"]
        # Remove /v1 suffix to get metrics endpoint
        metrics_url = base_url.replace("/v1", "/metrics")
        resp = requests.get(metrics_url, timeout=5)
        for line in resp.text.split("\n"):
            # vllm:kv_cache_usage_perc is the metric for KV cache usage
            if line.startswith("vllm:kv_cache_usage_perc"):
                value = float(line.split()[-1])
                return value * 100  # convert to percentage
    except (requests.RequestException, ValueError, IndexError):
        pass
    return -1

In [ ]:
if is_on_kaggle_interactive():
    test_prompt_ids = build_prompt_token_ids(
        system_content="Reply your answer in \\boxed{}",
        user_content="How many r are there in strawberry?",
        reasoning_effort=ReasoningEffort.HIGH,
    )
    resp: Completion = client.completions.create(
        model="vllm-model",
        prompt=test_prompt_ids,
        max_tokens=1024,
        temperature=1.0,
        extra_body=dict(
            min_p=0.02,
            stop_token_ids=stop_token_ids,
            return_token_ids=True,
        ),
    )

    print("Token IDs:", resp.choices[0].token_ids)  # type: ignore[attr-defined]

    print(resp.choices[0].text)

# Text processing

In [ ]:
def extract_boxed_text(text: str) -> str:
    """Extract text inside \\boxed{} from LaTeX-formatted text"""
    import re

    pattern: str = r"oxed{(.*?)}"
    matches: list[str] = re.findall(pattern, text)
    if not matches:
        return ""
    for match in matches[::-1]:
        if match != "":
            return match
    return ""


def is_valid_answer_string(text: str) -> bool:
    try:
        if int(text) == float(text):
            if 0 <= int(text) <= 99_999:
                # now AIMO answers no longer need modulo
                return True
    except Exception:
        pass
    return False

In [ ]:
from collections import Counter

completed_question_ids: set[str] = set()
question_id_to_counter: dict[str, Counter] = {"": Counter()}


import math
from collections import Counter


def vote_answer(question_id: str, force_answer: bool = False) -> int | None:
    # reads counter from global
    counter = question_id_to_counter[question_id]
    if force_answer and not counter:
        print(f"Current GPU usage {get_gpu_kv_cache_usage()}")
        print("force_answer=True but no answer recorded")
        completed_question_ids.add(question_id)
        return 12453

    # voting mechanism
    modified_counter: dict[int, float] = {}
    for value, count in counter.items():
        # re-weighted because smaller answers seems to be wrong
        # "1.25 +" because log(1) = 0
        modified_counter[value] = (
            modified_counter.get(value, 0.0) + math.log(1.25 + abs(value)) * count
        )

    total_score = sum(modified_counter.values())
    score_list = sorted(
        (score, counter[value], value) for value, score in modified_counter.items()
    )
    if force_answer:
        print(f"score_list | {total_score:8.1f} over {sum(counter.values())} attempts")
        print(f"Current GPU usage {get_gpu_kv_cache_usage()}")
        for score, count, value in score_list[::-1]:
            print(f"{value:10}   {score:8.1f} {count:8d}")
        return score_list[-1][-1]
    if score_list[-1][0] > max(1, total_score / (2 + math.log(1 + total_score))):
        if len(score_list) == 1:
            completed_question_ids.add(question_id)
        else:
            if score_list[-1][0] - score_list[-2][0] > 1:
                # win by a certain number of points at least
                completed_question_ids.add(question_id)
    return None

# Generate solution

In [ ]:
def generate_solution(
    question_text: str, question_id: str = "", solution_index: int = 0
) -> str:
    if question_id in completed_question_ids:
        return ""
    if time.time() >= cutoff_times[-1]:
        return ""

    try:
        # Build initial prompt as token IDs
        prompt_ids: list[int] = build_prompt_token_ids(
            system_content="You will solve the problem and return the final answer in \\boxed{}. The answer is expected to be an integer between 0 and 99999, inclusive. Do not guess the answer, unless specifically given permission to.",
            user_content=question_text,
            reasoning_effort=ReasoningEffort.HIGH,
        )

        all_token_ids: list[int] = prompt_ids.copy()
        generation_idx = 0

        for iteration in range(3):  # guess at 90, guess at 30
            # Inner loop
            while True:
                response_ids: list[int] = []
                text_response = ""
                breaking = False

                # Use streaming with completions API
                stream: Stream[Completion] = client.completions.create(
                    model="vllm-model",
                    prompt=prompt_ids,
                    max_tokens=32768,
                    temperature=1.0,
                    stream=True,
                    extra_body=dict(
                        min_p=0.02,
                        stop_token_ids=stop_token_ids,
                        return_token_ids=True,
                    ),
                )

                # Use StreamableParser to process streaming tokens
                stream_parser = StreamableParser(harmony_encoding, role=Role.ASSISTANT)

                for chunk in stream:
                    generation_idx += 1
                    # Get token IDs from the chunk (vLLM extension)
                    chunk_token_ids = getattr(chunk.choices[0], "token_ids", None)
                    if chunk_token_ids:
                        response_ids.extend(chunk_token_ids)
                        # Process tokens through harmony parser for text
                        for token_id in chunk_token_ids:
                            stream_parser.process(token_id)

                    # Also get text directly if available
                    chunk_text = chunk.choices[0].text
                    if chunk_text:
                        text_response += chunk_text

                    # Check finish_reason to see if generation completed naturally
                    finish_reason = chunk.choices[0].finish_reason
                    if finish_reason:
                        break

                    if question_id in completed_question_ids:
                        # stop generating if we have finalized on an answer
                        breaking = True
                    if time.time() >= cutoff_times[-1]:
                        breaking = True
                    if (
                        get_gpu_kv_cache_usage(question_id) > 70
                        and int(get_gpu_kv_cache_usage(question_id) + solution_index)
                        % num_generations
                        == 0
                    ):
                        print("terminated to prevent excessive GPU usage")
                        breaking = True
                    if breaking:
                        break
                    # instead of breaking = True, so we want to inject instructions for these conditions
                    if (
                        chunk_text
                        and "}" in chunk_text
                        and is_valid_answer_string(extract_boxed_text(text_response))
                    ):
                        break
                    if iteration == 0 and cutoff_times[-1] - time.time() < 90:
                        break
                    if iteration == 1 and cutoff_times[-1] - time.time() < 30:
                        break

                # Append response token IDs to prompt for multi-turn
                all_token_ids.extend(response_ids)
                stream.close()

                if breaking:
                    break

                # Exit inner loop
                break

            if breaking:
                break

            boxed_text = extract_boxed_text(text_response)
            user_follow_up = None
            print(
                f"solution {solution_index:02d}, iteration {iteration:02d} token {len(all_token_ids):05d}"
            )
            if not is_valid_answer_string(extract_boxed_text(text_response)):
                if iteration == 0 and cutoff_times[-1] - time.time() < 90:
                    print("follow-up - guess answer soon")
                    user_follow_up = "The answer is expected to be an integer between 0 and 99999 inclusive. Please make an educated guess (e.g. lower bound, upper bound, current best answer, ...) and put your your final answer in \\boxed{}."
                elif iteration == 1 and cutoff_times[-1] - time.time() < 30:
                    print("follow-up - guess answer now")
                    user_follow_up = "The answer is expected to be an integer between 0 and 99999 inclusive. Please guess a reasonable answer and put in \\boxed{} as soon as possible."
                else:
                    print("follow-up - ask boxed answer")
                    user_follow_up = "The answer is expected to be an integer between 0 and 99999 inclusive. Place your final answer in \\boxed{}. Do not guess the answer."
            elif int(boxed_text) <= 10:
                print("follow-up - are you sure")
                user_follow_up = (
                    "Are you sure that is the answer? Do not guess the answer."
                )
            elif iteration == 0 and len(all_token_ids) < 3200:
                print("follow-up - have you verified")
                user_follow_up = "Have you verified your answer?"
            else:
                # answer found, no issues detected, proceed to answering
                break

            if user_follow_up:
                # Append response and user follow-up as token IDs
                new_prompt_ids = append_user_turn_token_ids(
                    prompt_ids, response_ids, user_follow_up
                )
                # Track the new tokens added (user follow-up portion)
                added_tokens = new_prompt_ids[len(prompt_ids) + len(response_ids) :]
                all_token_ids.extend(added_tokens)
                prompt_ids = new_prompt_ids

        detokenized_text = harmony_encoding.decode(all_token_ids)
        boxed_text = extract_boxed_text(detokenized_text)

        if question_id and all_token_ids:
            answer_suffix = ""
            if is_valid_answer_string(boxed_text):
                answer_suffix = f"-{boxed_text}"
            total_tokens = len(all_token_ids)
            base_path = f"solutions/{question_id}/{solution_index:02d}-{total_tokens:05d}{answer_suffix}"
            # Save full stream as token IDs (one token ID per line)
            with open(f"{base_path}-tokens.txt", "w") as f:
                for token_id in all_token_ids:
                    f.write(f"{token_id}\n")
            # Save detokenized full stream for readability
            with open(f"{base_path}.txt", "w") as f:
                f.write(detokenized_text)

        if is_valid_answer_string(boxed_text):
            question_id_to_counter[question_id][int(boxed_text)] += 1
            vote_answer(question_id)

        return boxed_text

    finally:
        pass

In [ ]:
if is_on_kaggle_interactive():
    generate_solution("What is 1+1?")

In [ ]:
import concurrent.futures
from collections import Counter


def solve(question_text: str, question_id: str = "") -> int:
    print(f"processing {question_id}")
    await_client()
    print("client connected")
    os.makedirs(f"solutions/{question_id}", exist_ok=True)
    question_id_to_counter[question_id] = Counter()
    completed_question_ids.discard(question_id)  # just in case question_id collides

    if question_id and time.time() > cutoff_times[-1]:
        print("timeout did not solve")
        return 12314

    get_gpu_kv_cache_usage(
        question_id
    )  # run once to prevent running in the first batch of execution
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_generations) as executor:
        # run in parallel
        results = executor.map(
            generate_solution,
            [question_text] * num_generations,
            [question_id] * num_generations,
            list(range(num_generations)),
        )
        list(results)

    final_answer = vote_answer(question_id, force_answer=True)
    assert final_answer is not None
    return final_answer

In [ ]:
if is_on_kaggle_interactive():
    solve("What is 1+1?")

# Submission server

In [ ]:
import os

import kaggle_evaluation.aimo_3_inference_server
import pandas as pd
import polars as pl

if is_on_kaggle():
    pd.read_csv(
        "/kaggle/input/ai-mathematical-olympiad-progress-prize-3/reference.csv"
    ).drop("answer", axis=1).to_csv("reference.csv", index=False)


# Replace this function with your inference code.
# The function should return a single integer between 0 and 99999, inclusive.
def predict(id_: pl.Series, problem: pl.Series) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    # Unpack values
    question_id: str = id_.item(0)
    question_text: str = problem.item(0)

    if not run_all_questions:
        if is_on_kaggle_commit():
                if serve_vllm_on_kaggle:
                    # to conserve Kaggle H100 quota
                    if not("Norwegian" in question_text or "Alice" in question_text):
                        print("on kaggle commit, skipping question")  # not popping cutoff_times
                        return pl.DataFrame({"id": id_, "answer": 12315})
                else:
                    # to get quicker feedback
                    if not("Norwegian" in question_text or "Alice" in question_text):
                        print("on kaggle commit, skipping question")  # not popping cutoff_times
                        return pl.DataFrame({"id": id_, "answer": 12315})

        if not is_on_kaggle():
            # if you want to debug a particular question locally
            if not ("Alice" in question_text):
                print("not on kaggle, skipping question")  # not popping cutoff_times
                return pl.DataFrame({"id": id_, "answer": 12315})

    # Make a prediction
    prediction = solve(question_text, question_id=question_id)
    completed_question_ids.add(question_id)
    cutoff_times.pop()
    return pl.DataFrame({"id": id_, "answer": prediction})


inference_server = kaggle_evaluation.aimo_3_inference_server.AIMO3InferenceServer(
    predict  # type: ignore[arg-type]
)

print("Starting submission server")
if __name__ == "__main__":
    if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
        inference_server.serve()
    else:
        inference_server.run_local_gateway(("reference.csv",))